In [7]:
import numpy as np
import pandas as pd
import random 
import pickle
import json


In [2]:
import nltk

In [3]:
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

In [4]:
from tensorflow.keras.optimizers import SGD

In [15]:
lemmatizer = WordNetLemmatizer()

In [8]:
intents = json.loads(open('intents.json').read())

In [30]:
words=[]
classes=[]
documents=[]
ignore_letters = ['?', '!', '.', ',']

In [31]:
for intent in intents['intents'] :
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
        

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dheer\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [32]:
words

['hi',
 'hello',
 'how',
 'are',
 'you',
 '?',
 'howdy',
 'what',
 'is',
 'your',
 'name',
 'what',
 'can',
 'i',
 'call',
 'you',
 'can',
 'you',
 'tell',
 'me',
 'your',
 'name',
 'may',
 'i',
 'know',
 'farmer',
 'plans',
 'farmer',
 'plans',
 'planning',
 'financing',
 'plans',
 'I',
 'am',
 'going',
 'to',
 'die',
 'I',
 'need',
 'some',
 'emotional',
 'support',
 'please',
 'help',
 'me',
 'how',
 'can',
 'i',
 'navigate',
 'in',
 'you',
 '?',
 'how',
 'can',
 'i',
 'get',
 'full',
 'use',
 'from',
 'you',
 '?',
 'what',
 'can',
 'you',
 'do',
 '?',
 'thank',
 'you',
 'see',
 'you',
 '!',
 '!',
 'bye',
 'good',
 'bye']

In [26]:
documents

[(['hi'], 'greetings'),
 (['hello'], 'greetings'),
 (['how', 'are', 'you', '?'], 'greetings'),
 (['howdy'], 'greetings'),
 (['what', 'is', 'your', 'name'], 'name'),
 (['what', 'can', 'i', 'call', 'you'], 'name'),
 (['can', 'you', 'tell', 'me', 'your', 'name'], 'name'),
 (['may', 'i', 'know', 'farmer', 'plans'], 'plans'),
 (['farmer', 'plans'], 'plans'),
 (['planning'], 'plans'),
 (['financing', 'plans'], 'plans'),
 (['I', 'am', 'going', 'to', 'die'], 'emotional support'),
 (['I', 'need', 'some', 'emotional', 'support'], 'emotional support'),
 (['please', 'help', 'me'], 'emotional support'),
 (['how', 'can', 'i', 'navigate', 'in', 'you', '?'], 'use'),
 (['how', 'can', 'i', 'get', 'full', 'use', 'from', 'you', '?'], 'use'),
 (['what', 'can', 'you', 'do', '?'], 'use'),
 (['thank', 'you', 'see', 'you', '!', '!'], 'end'),
 (['bye'], 'end'),
 (['good', 'bye'], 'end'),
 (['hi'], 'greetings'),
 (['hello'], 'greetings'),
 (['how', 'are', 'you', '?'], 'greetings'),
 (['howdy'], 'greetings'),
 ([

In [33]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

In [19]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dheer\AppData\Roaming\nltk_data...


True

In [21]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dheer\AppData\Roaming\nltk_data...


True

In [34]:
words = sorted(set(words))

In [35]:
words

['I',
 'am',
 'are',
 'bye',
 'call',
 'can',
 'die',
 'do',
 'emotional',
 'farmer',
 'financing',
 'from',
 'full',
 'get',
 'going',
 'good',
 'hello',
 'help',
 'hi',
 'how',
 'howdy',
 'i',
 'in',
 'is',
 'know',
 'may',
 'me',
 'name',
 'navigate',
 'need',
 'plan',
 'planning',
 'please',
 'see',
 'some',
 'support',
 'tell',
 'thank',
 'to',
 'use',
 'what',
 'you',
 'your']

In [36]:
classes = sorted(set(classes))

In [37]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [38]:
training =[]
output_empty =[0]*  len(classes)

In [39]:
for document in documents:
    bag =[]
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag, output_row])

In [40]:
random.shuffle(training)
training=np.array(training)

C:\Users\dheer\AppData\Local\Temp\ipykernel_8720\4111924452.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [41]:
train_x = list(training[:, 0])
train_y = list(training[:,1])

In [43]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),) ,activation= 'relu'))

In [44]:
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='relu'))

In [46]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9)

In [47]:
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [57]:
hist = model.fit(np.array(train_x), np.array(train_y),epochs=200, batch_size=5, verbose=1)

Epoch 1/200
4/4 [==============================] - 1s 3ms/step - loss: 1.9046 - accuracy: 0.1500
Epoch 2/200
4/4 [==============================] - 0s 5ms/step - loss: 2.0601 - accuracy: 0.1000
Epoch 3/200
4/4 [==============================] - 0s 3ms/step - loss: 1.8990 - accuracy: 0.1000
Epoch 4/200
4/4 [==============================] - 0s 3ms/step - loss: 2.0423 - accuracy: 0.0500
Epoch 5/200
4/4 [==============================] - 0s 3ms/step - loss: 1.9309 - accuracy: 0.2500
Epoch 6/200
4/4 [==============================] - 0s 5ms/step - loss: 2.0235 - accuracy: 0.1500
Epoch 7/200
4/4 [==============================] - 0s 7ms/step - loss: 1.9867 - accuracy: 0.1000
Epoch 8/200
4/4 [==============================] - 0s 4ms/step - loss: 1.9312 - accuracy: 0.2000
Epoch 9/200
4/4 [==============================] - 0s 5ms/step - loss: 1.9396 - accuracy: 0.0000e+00
Epoch 10/200
4/4 [==============================] - 0s 5ms/step - loss: 1.9442 - accuracy: 0.0500
Epoch 11/200
4/4 [=======

In [58]:
model.save('chatbot_model.h5', hist)

INFO:tensorflow:Assets written to: chatbot_model.model\assets


In [50]:
#using the model

In [49]:
from tensorflow.keras.models import load_model

In [51]:
words = pickle.load(open('words.pkl', 'rb'))

In [52]:
classes =  pickle.load(open('classes.pkl', 'rb'))

In [54]:
model = load_model("chatbot_model.model")

In [63]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [64]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag =[0]*len(words)
    
    for w in sentence_words:
        for i, word in enumerate(words):
            if word==w:
                bag[i]=1
    return np.array(bag)

In [66]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res= model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD=0.25
    results =[[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    results.sort(key= lambda x:x[1], reverse=True)
    
    return_list =[]
    
    for r in results:
        return_list.append({'intent':classes[r[0]], 'probability': str(r[1])})
    return return_list
    

In [61]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents=intents_json['intents']
    for i in list_of_intents:
        if i['tag']==tag:
            result= random.choice(i['responses'])
            break
    return result

In [69]:
message = "can i use service"
ints = predict_class(message)
get_response(ints, intents)

1/1 [==============================] - 0s 46ms/step


'check our other services it might help you to go through some of your difficulties'

In [70]:
ans = get_response(ints, intents)

In [71]:
type(ans)

str

In [72]:
ans

'its okay there are lot of options out there you can try them'